In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras import applications
from keras.optimizers import SGD,Adam
from keras.optimizers import Adam
from keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] ='1'

In [3]:
# loading data
with open("X_train1.npy","rb") as fp:
    X = np.load(fp)
with open("y_train1.npy","rb") as fp:
    y = np.load(fp)

# model definition

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [8]:
def inc_models(img_rows, img_cols, channel=1, num_classes=None):
#     model = applications.resnet50.ResNet50(include_top=True, weights='imagenet')
    model = applications.inception_resnet_v2.InceptionResNetV2(include_top=True, weights='imagenet')
    # Truncate and replace softmax layer for transfer learning
    
    model.layers.pop()
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []
    
    new_model = Sequential()
    new_model.add(model)
    new_model.add(Dense(num_classes, activation='softmax'))

#     Uncomment below to set the first 10 layers to non-trainable (weights will not be updated)
    for layer in model.layers[:5]:
        layer.trainable = True

    # Learning rate is changed to 0.001
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    new_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return new_model

In [14]:
# def res_models(img_rows, img_cols, channel=1, num_classes=None):
#     model = applications.resnet50.ResNet50(include_top=True, weights='imagenet')

#     # Truncate and replace softmax layer for transfer learning
    
#     model.layers.pop()
#     model.outputs = [model.layers[-1].output]
#     model.layers[-1].outbound_nodes = []
    
#     new_model = Sequential()
#     new_model.add(model)
#     new_model.add(Dense(num_classes, activation='softmax'))

# #     Uncomment below to set the first 10 layers to non-trainable (weights will not be updated)
#     for layer in model.layers[:5]:
#         layer.trainable = True

#     # Learning rate is changed to 0.001
#     adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#     sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
#     new_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

#     return new_model

In [9]:
img_rows, img_cols = 299, 299 # Resolution of inputs
channel = 1
num_classes = 6
batch_size = 1
nb_epoch = 25

In [10]:
model = inc_models(img_rows, img_cols, channel, num_classes)

In [11]:
for i in model.layers:
    print(i,i.trainable)

<keras.engine.training.Model object at 0x7f89a6a272e8> True
<keras.layers.core.Dense object at 0x7f89a3b107b8> True


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  multiple                  54336736  
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 9222      
Total params: 54,345,958
Trainable params: 54,285,414
Non-trainable params: 60,544
_________________________________________________________________


In [13]:
# Start Fine-tuning
model.fit(X, y,
          nb_epoch=12,
          shuffle=True,
          verbose=1,
          validation_split=0.1
          )

/home/humonics/.virtualenvs/capi/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 5626 samples, validate on 626 samples
Epoch 1/12
5626/5626 [==============================] - 136s 24ms/step - loss: 0.7749 - acc: 0.7138 - val_loss: 0.3158 - val_acc: 0.8754
Epoch 2/12
5626/5626 [==============================] - 121s 22ms/step - loss: 0.2490 - acc: 0.9118 - val_loss: 0.2556 - val_acc: 0.9153
Epoch 3/12
5626/5626 [==============================] - 120s 21ms/step - loss: 0.1295 - acc: 0.9586 - val_loss: 0.1925 - val_acc: 0.9361
Epoch 4/12
5626/5626 [==============================] - 120s 21ms/step - loss: 0.0650 - acc: 0.9822 - val_loss: 0.2336 - val_acc: 0.9281
Epoch 5/12
5626/5626 [==============================] - 120s 21ms/step - loss: 0.0390 - acc: 0.9899 - val_loss: 0.2384 - val_acc: 0.9281
Epoch 6/12
5626/5626 [==============================] - 120s 21ms/step - loss: 0.0267 - acc: 0.9934 - val_loss: 0.2284 - val_acc: 0.9361
Epoch 7/12
5626/5626 [==============================] - 120s 21ms/step - loss: 0.0175 - acc: 0.9963 - val_loss: 0.2450 - val_acc: 0

In [14]:
model.save_weights('inc1.hdf5')
print("model saved..!")

model saved..!


In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  multiple                  54336736  
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 9222      
Total params: 54,345,958
Trainable params: 54,285,414
Non-trainable params: 60,544
_________________________________________________________________


In [22]:
ls

 AVGOT2.ipynb         ResNet50.hdf5                    train.zip*
'AV GOT.ipynb'*       result1.csv                      Xn_train.npy
 my_model.h5          result2.csv                      X_train1.npy
 my_model.hdf5        sample_submission_ns2btKE.csv*   X_train.npy*
 ResNet50_25c1.hdf5   test_ApKoW4T.csv*                yn_train.npy
 ResNet50_25.hdf5     train/                           y_train1.npy
 ResNet50.h5          train1.h5                        y_train.npy*


In [16]:
test = pd.read_csv('test_ApKoW4T.csv')

In [17]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('train/images/'+test['image'][i], target_size=(299,299,1))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 2680/2680 [00:04<00:00, 566.58it/s]


In [18]:
feature = model.predict(test)

In [19]:
results = []
for i in range(feature.shape[0]):
    pred = np.argmax(feature[i])
    results.append(pred)

In [20]:
len(results)

2680

In [21]:
test = pd.read_csv('test_ApKoW4T.csv')

## Generating Submission

In [22]:
sample = pd.read_csv('sample_submission_ns2btKE.csv')
sample['image'] = test['image']
sample['category'] = results
sample.to_csv('result_inc.csv', header=True, index=False)

In [29]:
ls

 AVGOT2.ipynb         result1.csv                      Xn_train.npy
'AV GOT.ipynb'*       result2.csv                      X_train1.npy
 my_model.h5          result3.csv                      X_train.npy*
 my_model.hdf5        sample_submission_ns2btKE.csv*   yn_train.npy
 ResNet50_25c1.hdf5   test_ApKoW4T.csv*                y_train1.npy
 ResNet50_25.hdf5     train/                           y_train.npy*
 ResNet50.h5          train1.h5
 ResNet50.hdf5        train.zip*


### saving to .npy

In [3]:
train = pd.read_csv('train/train.csv')

In [4]:
train.head(5)

,image,category
0,2823080.jpg,1
1,2870024.jpg,1
2,2662125.jpg,2
3,2900420.jpg,3
4,2804883.jpg,2


In [5]:
# We have grayscale images, so while loading the images we will keep grayscale=True, if you have RGB images, you should set grayscale as False
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('train/images/'+train['image'][i], target_size=(299,299,1))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 6252/6252 [00:09<00:00, 662.55it/s]


In [ ]:
!rm -f X_train.npy

In [11]:
ls

 AVGOT2.ipynb       ResNet50.hdf5                    train1.h5
'AV GOT.ipynb'*     result1.csv                      train.zip*
 my_model.h5        result2.csv                      Xn_train.npy
 my_model.hdf5      sample_submission_ns2btKE.csv*   X_train.npy*
 ResNet50_25.hdf5   test_ApKoW4T.csv*                yn_train.npy
 ResNet50.h5        train/                           y_train.npy*


In [5]:
np.save('X_train1.npy', X)

In [6]:
y=train['category'].values
y = to_categorical(y)

In [7]:
np.save('y_train1.npy', y)

In [30]:
pwd

'/data2/charan/api/AV/GOTDeepLearning'

In [22]:
ls

 AVGOT2.ipynb         result1.csv                      Xn_train.npy
'AV GOT.ipynb'*       result2.csv                      X_train1.npy
 my_model.h5          result3.csv                      X_train.npy*
 my_model.hdf5        sample_submission_ns2btKE.csv*   yn_train.npy
 ResNet50_25c1.hdf5   test_ApKoW4T.csv*                y_train1.npy
 ResNet50_25.hdf5     train/                           y_train.npy*
 ResNet50.h5          train1.h5
 ResNet50.hdf5        train.zip*


### Rough Work

In [3]:
train = pd.read_csv('train/train.csv')

In [18]:
len(train) #.head()

6252

In [19]:
list1 = []
list2 = []
list3 = []
list4 = []
list5 = []
for i in range(len(train['category'])):
    if train['category'][i] == 1:
        list1.append(train['image'][i])
    elif train['category'][i] == 2:
        list2.append(train['image'][i])
    elif train['category'][i] == 3:
        list3.append(train['image'][i])
    elif train['category'][i] == 4:
        list4.append(train['image'][i])
    else:
        list5.append(train['image'][i])

In [20]:
len(list1) + len(list2) + len(list3) + len(list4) + len(list5)

6252

In [14]:
len(list2)

1167

In [15]:
len(list3)

916

In [16]:
len(list4)

832

In [21]:
import shutil
import os

In [23]:
src_dir = '/data2/charan/api/AV/GOTDeepLearning/train/'

In [ ]:
for jpgfile in glob.iglob(os.path.join(src_dir, list)):
    shutil.copy(pngfile, dst_dir)